In [35]:
from docplex.mp.model import Model
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Clase instancia

class Instancia:
    def __init__(self, instance = "instancias/n20w120.001.txt"):
        self.instance_name = instance.split("/")[1]
        print(self.instance_name)
        # Parámetros
        self.M = 10000
        self.cust_no = None
        self.coord_x = None
        self.coord_y = None
        self.demand = None
        self.ready_time = None
        self.due_date = None
        self.service_time = None
        # Leer archivo
        self.problem = self.leer_archivo(instance)
    
    def leer_archivo(self,instance):
        # Lectura de instancias
        file_head = []
        cols_head = []
        instance_data = []
        with open(instance, "r") as f:
            file = f.readlines()
            file_head = file[0].replace("\n","").split("\t")[0].split(" ")
            file_head = "{} {} {}".format(file_head[1],file_head[5],file_head[6])
            cols_head = file[3].replace("\n","").split("\t")
            for line in file[6:-1]:
                splited_line = line.replace("\n","").split("\t")
                instance_data.append(splited_line)

        # Convertir en array
        instance_data = np.array(instance_data).astype(int)
        self.cust_no = instance_data[:,0]
        self.coord_x = instance_data[:,1]
        self.coord_y = instance_data[:,2]
        self.demand = instance_data[:,3]
        self.ready_time = instance_data[:,4]
        self.due_date = instance_data[:,5]
        self.service_time = instance_data[:,6]
        self.file_head = file_head
        
        
        self.V = instance_data[:,0]
        self.arcos = [(i,j) for i in self.V for j in self.V if i!=j]
        self.a = {i: instance_data[:,4][i-1] for i in self.V}
        self.b = {i: instance_data[:,5][i-1] for i in self.V}
        self.c = {(i,j): int(np.hypot(self.coord_x[i-1] - self.coord_x[j-1] , self.coord_y[i-1] - self.coord_y[j-1])) for i,j in self.arcos}
        
        

    def __repr__(self):
        return("Instancia: {} con {} clientes y header {}".format(self.instance_name,len(self.demand), self.file_head))

class Modelo():
    def __init__(self):
        self.name = "TSPTW"

    def build(self,ins):
        # Modelo
        self.mdl = Model('TSPTW')

        # Variables de decisión
        self.x = self.mdl.binary_var_dict(ins.arcos,name= 'x')
        self.u = self.mdl.integer_var_dict(ins.V, name= 'u', lb=0)

        # Restricción 1: Funcion Objetivo
        self.mdl.minimize(self.mdl.sum(self.x[(i,j)]*ins.c[(i,j)] for i,j in ins.arcos))

        # Restricción 2: Un arco saliente de cada vertice
        for i in ins.V:
            self.mdl.add_constraint(self.mdl.sum(self.x[(i,j)] for j in ins.V if i!=j) == 1, ctname = "restriccion2: x_%d" %(i))

        # Restricción 3: Un arco saliente de cada vertice
        for j in ins.V:
            self.mdl.add_constraint(self.mdl.sum(self.x[(i,j)] for i in ins.V if i!=j) == 1, ctname = "restriccion3: x_%d" %(j))

        # Restricción 4: Subrutas MTZ
        for i,j in ins.arcos:
            self.mdl.add_constraint(self.u[i]-self.u[j] + ins.M*self.x[(i,j)] <= ins.M - ins.c[(i,j)], ctname = "restriccion4: x_%d_%d" %(i,j))

        # # Restricción 5: Ventanas de Tiempo
        # for i in ins.V:
        #     self.mdl.add_constraints([ins.a[i] <= self.u[i] ,self.u[i] <= ins.b[i]])
        print(self.mdl.pprint_as_string())

    def solve(self):
        self.solucion = self.mdl.solve(log_output= True)

        print(self.mdl.get_solve_status())
        
        self.arcos_solucion = [i for i in ins.arcos if self.x[i].solution_value>0.9]

    def result(self):
        print(self.solucion)


    def plot(self,ins):
        fig, ax = plt.subplots(figsize=(12,7))
        for i in ins.V:
            if i!=0:
                ax.scatter(ins.coord_x[i],ins.coord_y[i],300,color="black",marker = 's',zorder=2)
                ax.annotate(str(i),xy=(ins.coord_x[i],ins.coord_y[i]),xytext=(ins.coord_x[i]-0.5,ins.coord_y[i]-0.5), color="white")
            else:
                ax.scatter(ins.coord_x[i],ins.coord_y[i],300,color="red",marker = 's',label='Depósito',zorder=2)
                ax.annotate(str(0),xy=(ins.coord_x[i],ins.coord_y[i]),xytext=(ins.coord_x[i]-0.5,ins.coord_y[i]-0.5), color="white")

        for i,j in self.arcos_solucion:
            plt.plot([ins.coord_x[i],ins.coord_x[j]],[ins.coord_y[i],ins.coord_y[j]],color='black',zorder=1)

        DPatch = mpatches.Patch(color='red',label='Deposito')
        ISolPatch = mpatches.Patch(color='black',label='Cliente')
        plt.legend(handles=[DPatch, ISolPatch])

        plt.xlabel("Coordenada x")
        plt.ylabel("Coordenada y")
        plt.title("Solución instancia")
        
        # plt.savefig("fig.png")
        plt.show()


ins = Instancia()
modelo = Modelo()
modelo.build(ins)
modelo.solve()
modelo.plot(ins)
#modelo.result()

n20w120.001.txt
// This file has been generated by DOcplex
// model name is: TSPTW
// var contrainer section
dvar bool x[420];
dvar int u[21];

minimize
 19 x_1_2 + 17 x_1_3 + 34 x_1_4 + 7 x_1_5 + 20 x_1_6 + 10 x_1_7 + 17 x_1_8
 + 29 x_1_9 + 15 x_1_10 + 24 x_1_11 + 29 x_1_12 + 23 x_1_13 + 29 x_1_14
 + 21 x_1_15 + 20 x_1_16 + 9 x_1_17 + 17 x_1_18 + 21 x_1_19 + 13 x_1_20
 + 12 x_1_21 + 19 x_2_1 + 10 x_2_3 + 42 x_2_4 + 27 x_2_5 + 3 x_2_6 + 27 x_2_7
 + 25 x_2_8 + 15 x_2_9 + 17 x_2_10 + 17 x_2_11 + 14 x_2_12 + 18 x_2_13
 + 49 x_2_14 + 17 x_2_15 + 6 x_2_16 + 21 x_2_17 + 14 x_2_18 + 17 x_2_19
 + 13 x_2_20 + 31 x_2_21 + 17 x_3_1 + 10 x_3_2 + 47 x_3_4 + 23 x_3_5
 + 13 x_3_6 + 26 x_3_7 + 15 x_3_8 + 25 x_3_9 + 23 x_3_10 + 26 x_3_11
 + 24 x_3_12 + 27 x_3_13 + 44 x_3_14 + 7 x_3_15 + 5 x_3_16 + 23 x_3_17
 + 21 x_3_18 + 25 x_3_19 + 18 x_3_20 + 30 x_3_21 + 34 x_4_1 + 42 x_4_2
 + 47 x_4_3 + 37 x_4_5 + 39 x_4_6 + 25 x_4_7 + 51 x_4_8 + 36 x_4_9 + 24 x_4_10
 + 27 x_4_11 + 38 x_4_12 + 25 x_4_13 + 44 x_4_14

DOcplexException: Model<TSPTW> did not solve successfully

In [34]:
print(ins.V)
print(ins.c)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
{(1, 2): 19, (1, 3): 17, (1, 4): 34, (1, 5): 7, (1, 6): 20, (1, 7): 10, (1, 8): 17, (1, 9): 29, (1, 10): 15, (1, 11): 24, (1, 12): 29, (1, 13): 23, (1, 14): 29, (1, 15): 21, (1, 16): 20, (1, 17): 9, (1, 18): 17, (1, 19): 21, (1, 20): 13, (1, 21): 12, (2, 1): 19, (2, 3): 10, (2, 4): 42, (2, 5): 27, (2, 6): 3, (2, 7): 27, (2, 8): 25, (2, 9): 15, (2, 10): 17, (2, 11): 17, (2, 12): 14, (2, 13): 18, (2, 14): 49, (2, 15): 17, (2, 16): 6, (2, 17): 21, (2, 18): 14, (2, 19): 17, (2, 20): 13, (2, 21): 31, (3, 1): 17, (3, 2): 10, (3, 4): 47, (3, 5): 23, (3, 6): 13, (3, 7): 26, (3, 8): 15, (3, 9): 25, (3, 10): 23, (3, 11): 26, (3, 12): 24, (3, 13): 27, (3, 14): 44, (3, 15): 7, (3, 16): 5, (3, 17): 23, (3, 18): 21, (3, 19): 25, (3, 20): 18, (3, 21): 30, (4, 1): 34, (4, 2): 42, (4, 3): 47, (4, 5): 37, (4, 6): 39, (4, 7): 25, (4, 8): 51, (4, 9): 36, (4, 10): 24, (4, 11): 27, (4, 12): 38, (4, 13): 25, (4, 14): 44, (4, 15): 54, (4, 16): 4

In [31]:
print(modelo.x)

{(1, 2): docplex.mp.Var(type=B,name='x_1_2'), (1, 3): docplex.mp.Var(type=B,name='x_1_3'), (1, 4): docplex.mp.Var(type=B,name='x_1_4'), (1, 5): docplex.mp.Var(type=B,name='x_1_5'), (1, 6): docplex.mp.Var(type=B,name='x_1_6'), (1, 7): docplex.mp.Var(type=B,name='x_1_7'), (1, 8): docplex.mp.Var(type=B,name='x_1_8'), (1, 9): docplex.mp.Var(type=B,name='x_1_9'), (1, 10): docplex.mp.Var(type=B,name='x_1_10'), (1, 11): docplex.mp.Var(type=B,name='x_1_11'), (1, 12): docplex.mp.Var(type=B,name='x_1_12'), (1, 13): docplex.mp.Var(type=B,name='x_1_13'), (1, 14): docplex.mp.Var(type=B,name='x_1_14'), (1, 15): docplex.mp.Var(type=B,name='x_1_15'), (1, 16): docplex.mp.Var(type=B,name='x_1_16'), (1, 17): docplex.mp.Var(type=B,name='x_1_17'), (1, 18): docplex.mp.Var(type=B,name='x_1_18'), (1, 19): docplex.mp.Var(type=B,name='x_1_19'), (1, 20): docplex.mp.Var(type=B,name='x_1_20'), (1, 21): docplex.mp.Var(type=B,name='x_1_21'), (2, 1): docplex.mp.Var(type=B,name='x_2_1'), (2, 3): docplex.mp.Var(type=B,